In [1]:
import cv2
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
 

# satellite APIs and geospatial libraries are not needed, since we start with numpy arrays
#import geopy.distance as distance
#import geopandas as gpd
#from shapely.geometry import Point

#import rioxarray
#from IPython.display import Image
#from PIL import Image as PILImage

#import planetary_computer as pc
#from pystac_client import Client
#import odc.stac

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import os

import image_modeling    # own file

#tf.compat.v1.enable_eager_execution()

In [3]:
RSEED = image_modeling.RSEED

NUM_LAYERS = image_modeling.NUM_LAYERS
IMAGE_SIZE = image_modeling.IMAGE_SIZE
IMG_SHAPE = image_modeling.IMG_SHAPE
NUM_TOTAL_VALUES = image_modeling.NUM_TOTAL_VALUES
DTYPE = image_modeling.DTYPE

In [4]:
# get path to the data folder
DATA_DIR = Path.cwd().parent.resolve() / "data"
assert DATA_DIR.exists()

MODEL_DIR = Path.cwd()

In [5]:
df = pd.read_csv(DATA_DIR / "dataset.csv")
df.head()

,uid,latitude,longitude,date,split,region,severity,density
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1,585.0
1,aacd,35.875083,-78.878434,2020-11-19,train,south,1,290.0
2,aaee,35.487000,-79.062133,2016-08-24,train,south,1,1614.0
3,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3,111825.0
4,aafl,39.474744,-86.898353,2021-08-23,train,midwest,4,2017313.0


In [6]:
# remove id's where no sat images could be collected
print("Initial entries in dataset: ", len(df))
errored_ids = ['einx', 'gygq', 'ifwc', 'jdvp', 'qpeh', 'tgiq', 'wrqa']
for id in errored_ids:
    df.drop(df.loc[df['uid']==id].index, inplace=True)
print("Remaining entries in dataset: ", len(df))
image_count = len(df)

Initial entries in dataset:  17060
Remaining entries in dataset:  17055


In [7]:
# add filepaths to dataframe
def get_path(id):
   return str(DATA_DIR/'sat_images/image_arrays_8_layer/') + f"/{id}.npy"

df['path'] = df["uid"].apply(get_path)

pd.set_option('display.max_colwidth', None)
df.head(5)

,uid,latitude,longitude,date,split,region,severity,density,path
0,aabm,39.080319,-86.430867,2018-05-14,train,midwest,1,585.0,/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/aabm.npy
1,aacd,35.875083,-78.878434,2020-11-19,train,south,1,290.0,/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/aacd.npy
2,aaee,35.487000,-79.062133,2016-08-24,train,south,1,1614.0,/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/aaee.npy
3,aaff,38.049471,-99.827001,2019-07-23,train,midwest,3,111825.0,/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/aaff.npy
4,aafl,39.474744,-86.898353,2021-08-23,train,midwest,4,2017313.0,/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/aafl.npy


In [8]:
# shuffle dataframe and create train/val/test splits
df_shuffled = df.sample(frac=1, random_state=RSEED, ignore_index=True)
df_train = df_shuffled.iloc[0:int(image_count * 0.8)].reset_index(drop=True)
df_eval   = df_shuffled.iloc[int(image_count * 0.8 ):int(image_count * 0.9)].reset_index(drop=True)
df_test  = df_shuffled.iloc[int(image_count * 0.9 ):].reset_index(drop=True)

print('Samples in train dataset: ', len(df_train))
print('Samples in evaluation dataset: ', len(df_eval))
print('Samples in test dataset: ', len(df_test))
print('Total samples: ', len(df_test) + len(df_train) + len(df_eval))

Samples in train dataset:  13644
Samples in evaluation dataset:  1705
Samples in test dataset:  1706
Total samples:  17055


In [9]:
# write .csv files with image paths and density (target value)
ALL_PATH   = MODEL_DIR / "file_list_all.csv"
TRAIN_PATH = MODEL_DIR / "file_list_train.csv"
EVAL_PATH  = MODEL_DIR / "file_list_eval.csv"
TEST_PATH  = MODEL_DIR / "file_list_test.csv"

df[['path','density']].to_csv(ALL_PATH, index=False, header=False)
df_train[['path','density']].to_csv(TRAIN_PATH, index=False, header=False)
df_eval[['path','density']].to_csv(EVAL_PATH, index=False, header=False)
df_test[['path','density']].to_csv(TEST_PATH, index=False, header=False)

In [10]:
test_path = DATA_DIR / 'sat_images/image_arrays_8_layer/aabm.npy'

In [11]:
#list_of_filepaths = [str(DATA_DIR/'sat_images/image_arrays_8_layer/') + f"/{id}.npy" for id in DATASET.uid.values]

In [12]:
#list_ds = tf.data.Dataset.list_files(str(DATA_DIR/'sat_images/image_arrays_8_layer/*.npy'), shuffle=False)
#list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [13]:
# # check that filenames are correct
# for f in list_ds.take(5):
#   print(f.numpy())

In [14]:
# def npy_header_offset(npy_path):
#     with open(str(npy_path), 'rb') as f:
#         if f.read(6) != b'\x93NUMPY':
#             raise ValueError('Invalid NPY file.')
#         version_major, version_minor = f.read(2)
#         if version_major == 1:
#             header_len_size = 2
#         elif version_major == 2:
#             header_len_size = 4
#         else:
#             raise ValueError('Unknown NPY file version {}.{}.'.format(version_major, version_minor))
#         header_len = sum(b << (8 * i) for i, b in enumerate(f.read(header_len_size)))
#         header = f.read(header_len)
#         if not header.endswith(b'\n'):
#             raise ValueError('Invalid NPY file.')
#         return f.tell()

In [15]:
# # for our files the header length is always 128 bytes
# header_offset = npy_header_offset(test_path)
# print(header_offset)

In [16]:
# def get_target_value(file_path):
#   # Convert the path to a list of path components
#   parts = tf.strings.split(file_path, os.path.sep)
#   # take the id from the filename
#   id = tf.strings.split(parts[-1], '.')[0]
#   # get the density value for this id from the dataset
#   target_value = DATASET.loc[DATASET['uid']==id]['density'].values[0]
#   return tf.constant(target_value, tf.float32)

In [17]:
## try out the function
#get_target_value(test_path).numpy()

In [18]:
# tf_dataset = tf.data.FixedLengthRecordDataset(
#     list_of_filepaths, 
#     NUM_TOTAL_VALUES * DTYPE.size, 
#     header_bytes=header_offset, 
#     )

In [19]:
#tf.reshape(tf.io.decode_raw(img, DTYPE), IMG_SHAPE)

In [20]:
# # maybe do this after batching ???

# tf_dataset = tf_dataset.map(lambda s: 
# tf_dataset = tf_dataset.map

In [21]:
#tfimg = tf.io.read_file(test_path)

In [22]:
#tfimg = tf.io.decode_raw(tfimg, tf.float32)[32:]

In [23]:
#tfimg = tf.reshape(tfimg, [NUM_LAYERS, IMAGE_SIZE, IMAGE_SIZE])

In [24]:
#tfimg.shape

In [25]:
#print(tfimg[32:]) # npy value

In [26]:
# dataset = tf.data.FixedLengthRecordDataset(
#         filenames=[test_path],
#         record_bytes=NUM_TOTAL_VALUES * DTYPE.size, 
#         header_bytes=128
#     )
# image_bytes = dataset.as_numpy_iterator().next()

In [27]:
#image_bytes

In [28]:
# datatype_bits = 32
# remove_len = 1024//datatype_bits
# tfimg = tfimg[remove_len:]

In [29]:
# image = tf.reshape(tf.io.decode_raw(image_bytes, DTYPE), IMG_SHAPE,)

In [30]:
# iterator = tf_dataset.take(1).as_numpy_iterator()
# object = iterator.next()
# print(object.shape)

In [31]:
# # RGB image from individual channels and masked image from masked channels
# rgb_image = np.array(
#         [object[0],
#          object[1],
#          object[2]]
#         )
# masked_image = np.array(
#         [object[4],
#          object[5],
#          object[6]]
#         )
# # show normalized image (range shifted to approximately 0-255, no hard limit though)
# plt.figure(figsize=(14,6))
# plt.subplot(121)
# plt.imshow(np.transpose((rgb_image), axes=[1, 2, 0]));
# plt.subplot(122)
# plt.imshow(np.transpose((masked_image), axes=[1, 2, 0]));

In [32]:
1024//32

32

In [33]:
# print(type(img))
# print(img.get_shape())
# np_array = tf.io.decode_raw(img, DTYPE, ).numpy()
# np_array.reshape((NUM_LAYERS, IMAGE_SIZE, IMAGE_SIZE))

In [34]:
# def decode_img(img):
#   # Convert the compressed string to a 3D uint8 tensor
#   img = tf.io.decode_jpeg(img, channels=3)
#   # Resize the image to the desired size
#   return tf.image.resize(img, [img_height, img_width])

In [35]:
# def process_path(file_path):
#   target = get_target_value(file_path)
#   # Load the raw data from the file as a string
#   img = tf.io.read_file(file_path)
#   img = decode_img(img)
#   return img, target

## Modeling

In [36]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [37]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [38]:
# Check for Tensorflow version
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

2.8.0


In [39]:
# Import variables from image_modelling.py file
HEIGHT = image_modeling.HEIGHT
WIDTH = image_modeling.WIDTH
BATCH_SIZE = image_modeling.BATCH_SIZE
TRAINING_SIZE = len(df_train) #!wc -l < flowers_train.csv
#TRAINING_STEPS = int(TRAINING_SIZE[0]) // BATCH_SIZE
TRAINING_STEPS = TRAINING_SIZE // BATCH_SIZE

In [40]:
# Lets create a simple linear model.
def linear_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(
        input_shape=IMG_SHAPE, 
        #input_shape=[HEIGHT, WIDTH, 3], 
        name='image'
        ))
    model.add(tf.keras.layers.Flatten(data_format="channels_last"))
    # We want to have a simple linear model so we have 
    # no activation function. 
    model.add(tf.keras.layers.Dense(units=1, activation=None))
    return model

In [41]:
# Create a function for training and evaluation
def train_and_evaluate(model, batch_size):
    
    model.compile(
        optimizer="adam", 
        ##loss=tf.keras.losses.MeanSquaredError(),
        #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        loss='mse',
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
    
    dataset = image_modeling.load_dataset(TRAIN_PATH, batch_size)
    eval_dataset = image_modeling.load_dataset(EVAL_PATH, batch_size, training=False)
    
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.fit(
        dataset, 
        validation_data=eval_dataset,
        #steps_per_epoch=TRAINING_STEPS, 
        epochs=10,
        callbacks=[tensorboard_callback]
    )
    
    
    return model

In [42]:
# Write a testing function.
def test(model):
    
    test_dataset = image_modeling.load_dataset(TEST_PATH, batch_size=1, training=False)
    model.evaluate(test_dataset)

### some testing

In [43]:
# # check if loading works correctly
# dataset1 = image_modeling.load_dataset(TRAIN_PATH, BATCH_SIZE)
# elem = next(iter(dataset1))
# print(elem[0][0][0].shape)
# elem[1][5].numpy()

In [44]:

# rgb_image = np.array(
#         [elem[0][17][0],
#          elem[0][17][1],
#          elem[0][17][2]]
#         )
# # show normalized image (range shifted to approximately 0-255, no hard limit though)
# plt.figure(figsize=(14,6))
# plt.subplot(121)
# plt.imshow(np.transpose((rgb_image), axes=[1, 2, 0]));
# #plt.subplot(122)
# #plt.imshow(np.transpose((masked_image), axes=[1, 2, 0]));

In [45]:
# it = iter(df[df["date"] > "2015-07-01"].uid.values)

In [46]:
# id = next(it)
# filepath = f'/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/{id}.npy'
# loaded_layers = np.load(filepath)
# water_pixels = (loaded_layers[4] > 0.0).sum()
# mean_r  = loaded_layers[4].sum()/(water_pixels) if (water_pixels > 0) else 0.0
# mean_g  = loaded_layers[5].sum()/(water_pixels) if (water_pixels > 0) else 0.0
# mean_b  = loaded_layers[6].sum()/(water_pixels) if (water_pixels > 0) else 0.0
# mean_ir = loaded_layers[7].sum()/(water_pixels) if (water_pixels > 0) else 0.0
# print('ID: ', id)
# print('Max value in masked image: ', loaded_layers[4:].max())
# print('Water pixels: ', water_pixels)
# print('Density: ', df[df['uid'] == id].density.values[0])
# print('Mean R:  ', mean_r)
# print('Mean G:  ', mean_g)
# print('Mean B:  ', mean_b)
# print('Mean IR: ', mean_ir)

# # RGB image from individual channels and masked image from masked channels
# rgb_image = np.array([loaded_layers[0],loaded_layers[1],loaded_layers[2]])
# masked_image = np.array([loaded_layers[4],loaded_layers[5],loaded_layers[6]])
# #hsv_image = cv2.cvtColor(np.transpose((loaded_layers[:3]), axes=[1, 2, 0]), cv2.COLOR_RGB2HSV)
# hsv_image = np.transpose(cv2.cvtColor(np.transpose((rgb_image), axes=[1, 2, 0]), cv2.COLOR_RGB2HSV), axes=[2, 0, 1])
# # show normalized image (range shifted to approximately 0-255, no hard limit though)
# plt.figure(figsize=(14,6))
# plt.subplot(121)
# plt.imshow(np.transpose((rgb_image), axes=[1, 2, 0]));
# plt.subplot(122)
# plt.imshow(np.transpose((hsv_image), axes=[1, 2, 0]));
# # plot hue layer
# plt.imshow(hsv_image[0], cmap='gray');


In [47]:
# (hsv_image[0]/360).max()

In [48]:
# cv2.cvtColor(np.transpose((loaded_layers[:3]), axes=[1, 2, 0]), cv2.COLOR_RGB2HSV)

In [49]:
# # Try to load one of the saved files to see if everything is working
# filepath = '/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/pwcw.npy'
# loaded_layers = np.load(filepath)
# print(loaded_layers[4:].shape)

# # # RGB image from individual channels and masked image from masked channels
# # rgb_image = np.array(
# #         [loaded_layers[0],
# #          loaded_layers[1],
# #          loaded_layers[2]]
# #         )
# # masked_image = np.array(
# #         [loaded_layers[4],
# #          loaded_layers[5],
# #          loaded_layers[6]]
# #         )
# # show normalized image (range shifted to approximately 0-255, no hard limit though)
# # plt.figure(figsize=(14,6))
# # plt.subplot(121)
# # plt.imshow(np.transpose((rgb_image), axes=[1, 2, 0]));
# #plt.subplot(122)
# #plt.imshow(np.transpose((masked_image), axes=[1, 2, 0]));

In [50]:
# def decode_dataset(data_row):
#     record_defaults = ['path', 'target']#tf.constant([0.0], dtype=tf.float32)]
#     filename, target = tf.io.decode_csv(data_row, record_defaults)
#     #np_image_layers = np.load(filename)
#     #image_tensor = tf.convert_to_tensor(np_image_layers, np.float32)
#     #
#     #image_bytes = tf.io.read_file(filename=filename)
#     #label = tf.math.equal(label_string, CLASS_NAMES)
#     #return image_tensor, target
#     return filename, target

# def decode_dataset(data_row):
#     record_defaults = ['path', tf.constant([0.0], dtype=tf.float32)]
#     filename, target = tf.io.decode_csv(data_row, record_defaults)
    
#     image_bytes = tf.data.FixedLengthRecordDataset(
#         filenames=[filename],
#         record_bytes=NUM_TOTAL_VALUES * DTYPE.size, 
#         header_bytes=128
#     )
#     #image_bytes = tf.io.read_file(filename=filename)
#     ## remove header bytes # (32 bytes for float32, 16 bytes for float64, 64 bytes for float16)
#     #image_bytes = image_bytes[32:]

#     #np_image_layers = np.load(filename)
#     #image_tensor = tf.convert_to_tensor(np_image_layers, np.float32)
#     #label = tf.math.equal(label_string, CLASS_NAMES)
#     return image_bytes, target

In [51]:
#import image_modeling

In [52]:
#np.load(test_path)

In [53]:
# dataset = tf.data.TextLineDataset(filenames=TRAIN_PATH)
# dataset = dataset.map(image_modeling.decode_dataset)
# #dataset = dataset.map(decode_dataset)

In [54]:
#image_bytes = tf.io.read_file(test_path)
# remove header bytes # (32 bytes for float32, 16 bytes for float64, 64 bytes for float16)
#image_bytes = image_bytes[32:]

In [55]:
#image_bytes.numpy().

In [56]:
#image_bytes[32:]

In [57]:
#image_bytes.shape

In [58]:
# # Load the numpy files
# def numpy_loader(filename):
#   image_array = np.load(str(filename))
#   return image_array

# def wrapper_func(filename, target):
#     img = tf.numpy_function(numpy_loader, [filename], tf.float32)
#     return img, target

In [59]:
# len(test_path)

In [60]:
# test = "b'/Users/markus/neuefische/tick-tick-bloom/data/sat_images/image_arrays_8_layer/qone.npy'"
# test[2:-1]

In [61]:
# type(np.load(test_path))

In [62]:
# tf.numpy_function(numpy_loader, [test_path], tf.float32)

In [63]:
## Build and train our model using the prior defined functions 
#model = linear_model()
#trained_model = train_and_evaluate(model, BATCH_SIZE)

In [64]:
%tensorboard --logdir logs/fit

In [65]:
# Call the testing function for our model
#test(model)

In [66]:
# # Now let us move on to a CNN model. 
def cnn_model():

    model = tf.keras.models.Sequential()
    #model.add(tf.keras.layers.InputLayer(
    #                                    input_shape=IMG_SHAPE,
                                        #data_format="channels_first",  
                                        #input_shape=[HEIGHT, WIDTH, 3], 
                                        #name='image'
    #                                    ))
    model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=[5, 5], padding="same", activation="relu", input_shape=IMG_SHAPE, data_format="channels_first"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    # model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=[5, 5], padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    # model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=[5, 5], padding="same", activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    # model.add(tf.keras.layers.Dropout(0.25))
    # model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=[5, 5], padding="same", activation="relu"))
    # model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    # # model.add(tf.keras.layers.Dropout(0.25))
    # model.add(tf.keras.layers.Conv2D(filters=20, kernel_size=[5, 5], padding="same", activation="relu"))
    # model.add(tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2))
    # # model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=300, activation="relu"))#, kernel_regularizer=tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dense(units=1, activation=None))

    # model = tf.keras.Sequential()
    # #model.add(tf.keras.layers.InputLayer(input_shape=IMG_SHAPE, name='image'))
    # model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=[3, 3], activation='relu', input_shape=IMG_SHAPE, data_format='channels_first'))#, padding='same'))
    # # model.add(tf.keras.layers.MaxPooling2D(2))
    # model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=[3, 3], activation='relu', padding='same'))
    # model.add(tf.keras.layers.MaxPooling2D(2))
    # model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=[3, 3], activation='relu', padding='same'))
    # # model.add(tf.keras.layers.MaxPooling2D(2))
    # model.add(tf.keras.layers.Conv2D(filters=9, kernel_size=[3, 3], activation='relu', padding='same'))
    # model.add(tf.keras.layers.MaxPooling2D(2))
    # model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=[3, 3], activation='relu', padding='same'))
    # # model.add(tf.keras.layers.MaxPooling2D(2))
    # model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=[3, 3], activation='relu', padding='same'))
    # model.add(tf.keras.layers.MaxPooling2D(2))
    # model.add(tf.keras.layers.Flatten())
    # model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    # model.add(tf.keras.layers.Dense(units=1))
    return model


In [67]:
model = cnn_model()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-02-03 19:06:11.258823: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-03 19:06:11.258981: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 10, 128, 128)      1010      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 64, 128)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 64, 20)         64020     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 32, 20)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 32, 20)         10020     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 16, 20)        0

In [69]:
# Let us fit the convolutional neural network
trained_model = train_and_evaluate(model, BATCH_SIZE)

Epoch 1/10


2023-02-03 19:06:11.579327: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-03 19:06:11.907321: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


    427/Unknown - 48s 112ms/step - loss: 54205209378816.0000 - root_mean_squared_error: 7362419.0000

2023-02-03 19:07:00.480596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


427/427 [==============================] - 50s 117ms/step - loss: 54205209378816.0000 - root_mean_squared_error: 7362419.0000 - val_loss: 7466964221952.0000 - val_root_mean_squared_error: 2732574.5000
Epoch 2/10
427/427 [==============================] - 49s 115ms/step - loss: 53895480999936.0000 - root_mean_squared_error: 7341354.0000 - val_loss: 7472164634624.0000 - val_root_mean_squared_error: 2733526.0000
Epoch 3/10
427/427 [==============================] - 49s 115ms/step - loss: 53894373703680.0000 - root_mean_squared_error: 7341279.0000 - val_loss: 7471239266304.0000 - val_root_mean_squared_error: 2733356.7500
Epoch 4/10
427/427 [==============================] - 50s 116ms/step - loss: 53893127995392.0000 - root_mean_squared_error: 7341193.5000 - val_loss: 7469775454208.0000 - val_root_mean_squared_error: 2733089.0000
Epoch 5/10
427/427 [==============================] - 50s 116ms/step - loss: 53891936813056.0000 - root_mean_squared_error: 7341112.5000 - val_loss: 7468068896768.

In [70]:
test(model)

1706/1706 [==============================] - 12s 7ms/step - loss: 28885919989760.0000 - root_mean_squared_error: 5374562.5000
